In [1]:
import numpy as np 
import pandas as pd

In [2]:
athlete_df = pd.read_csv('/Users/meetsudra/Documents/GitHub/olympics_dataanalysis/athlete_events.csv')
region_df = pd.read_csv('/Users/meetsudra/Documents/GitHub/olympics_dataanalysis/noc_regions.csv')

In [3]:
athlete_df.shape

(271116, 15)

### setting up the dataset to see it on the webpage

In [4]:
athlete_df = athlete_df[athlete_df['Season'] == 'Summer']

In [5]:
region_df.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [6]:
df = athlete_df.merge(region_df,on='NOC',how='left')
# we want region name of each player thus applying left join

In [7]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN
4,8,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,NaN,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,NaN,Netherlands,NaN


In [8]:
df['region'].unique().shape # total countries participated so far

(206,)

In [9]:
# checking missing and duplicate values
df.isnull().sum()

ID             0
Name           0
Sex            0
Age         9189
Height     51857
Weight     53854
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     188464
region       370
notes     218151
dtype: int64

In [10]:
df.duplicated().sum()
df.drop_duplicates(inplace=True)

# Medal_tally

### Getting data ready for medal-tally option

In [11]:
df['Medal'].value_counts()

Medal
Gold      11456
Bronze    11409
Silver    11212
Name: count, dtype: int64

In [12]:
df = pd.concat([df,pd.get_dummies(df['Medal'])],axis=1) # one-hot encoding 
df.shape

(221167, 20)

In [14]:
medal_tally = df.drop_duplicates(subset=['Team','NOC','Games','Year','City','Sport','Event','Medal'])
medal_tally.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes,Bronze,Gold,Silver
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN,False,False,False
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN,False,False,False
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN,False,False,False
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN,False,True,False
4,8,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,NaN,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,NaN,Netherlands,NaN,False,False,False


In [15]:
medal_tally = medal_tally.groupby('region').sum()[["Gold","Silver","Bronze"]].sort_values('Gold',ascending=False).reset_index()

# problem : here data is not exactly matching with real world data. there is a slight variation this might be due to the exception listed in the wikipedia document below accurate result tables.

In [17]:
medal_tally['total'] = medal_tally["Gold"] + medal_tally["Silver"] + medal_tally["Bronze"]
medal_tally

,region,Gold,Silver,Bronze,total
0,USA,1035,802,708,2545
1,Russia,592,498,487,1577
2,Germany,444,457,491,1392
3,UK,278,317,300,895
4,France,234,256,287,777
...,...,...,...,...,...
200,Lesotho,0,0,0,0
201,Albania,0,0,0,0
202,Libya,0,0,0,0
203,Liechtenstein,0,0,0,0


### Sidebar drop downs in Medal-tally section

In [18]:
# which year olympics was played
years = df['Year'].unique().tolist()
years.sort()


In [19]:
years.insert(0,'overall')
years

['overall',
 1896,
 1900,
 1904,
 1906,
 1908,
 1912,
 1920,
 1924,
 1928,
 1932,
 1936,
 1948,
 1952,
 1956,
 1960,
 1964,
 1968,
 1972,
 1976,
 1980,
 1984,
 1988,
 1992,
 1996,
 2000,
 2004,
 2008,
 2012,
 2016]

In [20]:
country = np.unique(df['region'].dropna().values).tolist()

In [21]:
country.insert(0,'overall')

### Year and country wise selection 

In [22]:
medal_df = df.drop_duplicates(subset=['Team','NOC','Games','Year','City','Sport','Event','Medal'])

In [60]:
# function to make dropdowns work

def fetch_medal_tally(year, country):

    flag = 0   # by-default flag
    if year == 'overall' and country == 'overall':
        temp_df = medal_df

    if year == 'overall' and country != 'overall':
        flag = 1
        temp_df = medal_df[medal_df['region'] == country]

    if year != 'overall' and country == 'overall':
        temp_df = medal_df[medal_df['Year'] == int(year)]

    if year != 'overall' and country != 'overall':
        temp_df = medal_df[(medal_df['Year'] == int(year)) & (medal_df['region'] == country)]

    if flag == 1:
        x = temp_df.groupby('Year').sum()[['Gold', 'Silver', 'Bronze']].sort_values('Year', ascending=True).reset_index()
    else :
        x = temp_df.groupby('region').sum()[['Gold', 'Silver', 'Bronze']].sort_values('Gold', ascending=False).reset_index()
        x['total'] = x['Gold'] + x['Silver'] + x['Bronze']

    print(x)


In [61]:
fetch_medal_tally(year='overall', country='India')

    Year  Gold  Silver  Bronze
0   1900     0       2       0
1   1920     0       0       0
2   1924     0       0       0
3   1928     1       0       0
4   1932     1       0       0
5   1936     1       0       0
6   1948     1       0       0
7   1952     1       0       1
8   1956     1       0       0
9   1960     0       1       0
10  1964     1       0       0
11  1968     0       0       1
12  1972     0       0       1
13  1976     0       0       0
14  1980     1       0       0
15  1984     0       0       0
16  1988     0       0       0
17  1992     0       0       0
18  1996     0       0       1
19  2000     0       0       1
20  2004     0       1       0
21  2008     1       0       2
22  2012     0       2       4
23  2016     0       1       1
